In [1]:
# Hidden cell for nbsphinx (check metadata)

#{
#  "nbsphinx": "hidden",
#  "trusted": false
#}

import warnings

warnings.simplefilter('ignore')

# Quickstart for Carsus

## Initialization and imports

In [2]:
import os
import carsus
from carsus.io.nist import NISTWeightsComp, NISTIonizationEnergies
from carsus.io.kurucz import GFALLReader
from carsus.io.zeta import KnoxLongZeta
from carsus.io.output import TARDISAtomData

In [3]:
CARSUS_PATH = os.path.dirname(carsus.__file__)

In [4]:
# Choose an output filename
output_file = 'kurucz_H-Zn.h5'

## Atomic weights and ionization energies

Get atomic weights and ionization energies from NIST. For example, to choose all elements from `H` to `Zn`:

In [5]:
atomic_weights = NISTWeightsComp('H-Zn')
ionization_energies = NISTIonizationEnergies('H-Zn')

Remember _Carsus_ allows you to pick ions in many ways!

- **Grab only some species:**

```python
ionization_energies = NISTIonizationEnergies('C 2-3')
```

- **Non consecutive ions:**

```python
ionization_energies = NISTIonizationEnergies('C, O, Si')
```

- **A mix of all above:**

```python
ionization_energies = NISTIonizationEnergies('H, C 1-2, Ar-Zn')
```

Once you are done, save data to the HDF5 output.

In [6]:
atomic_weights.to_hdf(output_file)
ionization_energies.to_hdf(output_file)

## Levels and lines

### GFALL

Download `gfall.dat` and create a `gfall_reader` instance:

In [7]:
!wget -q http://kurucz.harvard.edu/linelists/gfall/gfall.dat

In [8]:
gfall_reader = GFALLReader('gfall.dat')

[carsus.io.kurucz.gfall][WARNING]  A specific combination to identify unique levels from the gfall data has not been given. Defaulting to ["energy", "j"]. (gfall.py:68)


Then create a `TARDISAtomData` object passing the `gfall_reader`, `ionization_energies` and the selected ions:

In [9]:
atom_data = TARDISAtomData(gfall_reader, 
                           ionization_energies, 
                           ions='H-Zn')

[carsus.io.kurucz.gfall][INFO   ]  Parsing GFALL gfall.dat (gfall.py:116)
[carsus.io.kurucz.gfall][INFO   ]  Extracting line data: atomic_number, ion_charge, energy_lower, j_lower, energy_upper, j_upper, wavelength, loggf (gfall.py:292)


## Zeta data

In [10]:
zeta = KnoxLongZeta(os.path.join(CARSUS_PATH, 'data/knox_long_recombination_zeta.dat'))

In [11]:
zeta.to_hdf(output_file)

## Output HDF5 files

Finally, dump the `atom_data` object attributes to the HDF5 file.

In [12]:
atom_data.to_hdf(output_file)

Signing AtomData: 
MD5: 37ee6b2bede07756ce9b02c48c4f79af
UUID1: ded1cf20831d11eaaedac83dd46c2c53
